In [ ]:
%env VECLIB_MAXIMUM_THREADS=1
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

!make clean

import os, platform
import numpy as np
#np.show_config()

# SIMD: vector processing

1. Types of parallelism.
2. x86 intrinsic funcions.
3. Inspect assembly.

# Types of parallelism

The popular computer architecture is based on sequential processing.  The most fundamental processing unit executes instructions one by one.

If we assume the processor can only perform sequantial processing, we need to use multiple processors to perform parallel processing.  Differentiated by the memory access, the parallelism can be broadly set to two categories:

* Shared-memory parallel processing
* Distributed-memory parallel processing

# Vector processing

When the parallelism happens in the processor (one processing unit or core), it is usually done once for a single instruction with multiple data (SIMD).  It has also been called vector processing.  Vector processing is an illustrative name.

# Check CPU capabilities

x86 provides a series of SIMD instructions, including

* 64-bit: MMX
* 128-bit: SSE, SSE2, SSE3, SSE4, SSE4.1, SSE4.2 (streaming simd extension)
* 256-bit: AVX, AVX2 (advanced vector extension)
* 512-bit: AVX-512

Recent processors usually are equipped with AVX2, which was released with Haswell in 2013.  Before asking the compiler to use the specific instruction set, query the operating system for the cpu capabilities.

In [ ]:
print("Check on", platform.system())
if 'Linux' == platform.system():
    # check whether your cpu supports avx2 on linux
    !grep flags /proc/cpuinfo
elif 'Darwin' == platform.system():
    # check whether your cpu supports avx2 on mac
    !sysctl -a | grep machdep.cpu.*features

# x86 intrinsic functions

Major compilers provide header files for using the intrinsic functions that can be directly translated into the SIMD instructions:

* `<mmintrin.h>`: MMX
* `<xmmintrin.h>`: SSE
* `<emmintrin.h>`: SSE2
* `<pmmintrin.h>`: SSE3
* `<tmmintrin.h>`: SSSE3
* `<smmintrin.h>`: SSE4.1
* `<nmmintrin.h>`: SSE4.2
* `<ammintrin.h>`: SSE4A
* `<immintrin.h>`: AVX
* `<zmmintrin.h>`: AVX512

You may also use `<x86intrin.h>` which includes everything.

The first example, `01_mul/mul.cpp`, shows how to use the 256-bit-wide AVX to perform vector multiplication for 8 single-precision floating-point values.

```cpp
constexpr const size_t width = 8;
constexpr const size_t repeat = 1024 * 1024;
constexpr const size_t nelem = width * repeat;
```

In [ ]:
# time the difference between the loop and the simd/avx versions
!g++ -std=c++17 -g -O3 -m64 -mavx 01_mul/mul.cpp -o 01_mul/mul ; 01_mul/mul

## Inspect the assembly

I use radare2 to inspect the assembly of the generated image.

In [ ]:
# take a look at the symbol table
!r2 -Aqc "e scr.color=0 ; afl" 01_mul/mul

### 1 multiplication

To demonstrate the effect of different ratio of calculations to memory access, I use 3 sets of multiplication.  The first set uses 1 multiplication:

```cpp
void multiply1_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * b[j];
        }
    }
}

void multiply1_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *mb);
    }
}
```

In [ ]:
# 1 multiplication with loop
!r2 -Aqc "e scr.color=0 ; s sym.multiply1_loop_float__float__float ; pdf" 01_mul/mul

In [ ]:
# 1 multiplication with simd/avx
!r2 -Aqc "e scr.color=0 ; s sym.multiply1_simd_float__float__float ; pdf" 01_mul/mul

### 3 multiplication

The second set uses 3 multiplications:

```cpp
void multiply3_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * a[j];
            r[j] *= b[j];
            r[j] *= b[j];
        }
    }
}

void multiply3_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *ma);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
    }
}
```

In [ ]:
# 3 multiplication with loop
!r2 -Aqc "e scr.color=0 ; s sym.multiply3_loop_float__float__float ; pdf" 01_mul/mul

In [ ]:
# 3 multiplication with simd/avx
!r2 -Aqc "e scr.color=0 ; s sym.multiply3_simd_float__float__float ; pdf" 01_mul/mul

### 5 multiplication

The third (last) set uses 5 multiplications:

```cpp
void multiply5_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * a[j];
            r[j] *= a[j];
            r[j] *= b[j];
            r[j] *= b[j];
            r[j] *= b[j];
        }
    }
}

void multiply5_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *ma);
        *mr = _mm256_mul_ps(*mr, *ma);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
    }
}
```

In [ ]:
# 5 multiplication with loop
!r2 -Aqc "e scr.color=0 ; s sym.multiply5_loop_float__float__float ; pdf" 01_mul/mul

In [ ]:
# 5 multiplication with simd/avx
!r2 -Aqc "e scr.color=0 ; s sym.multiply5_simd_float__float__float ; pdf" 01_mul/mul

## Intel intrinsics guide

Intel maintains a website to show the available intrinsics: https://software.intel.com/sites/landingpage/IntrinsicsGuide/ .  Consult and remember it when needed.

# Exercises

1. Replace the single-precision floating-point vector type `__m256` with the double-precision floating-point vector type `__m256d` in the example, and compare the performance with the sinple-precision version.

# References

1. Crunching Numbers with AVX and AVX2 (AVX tutorials): https://www.codeproject.com/Articles/874396/Crunching-Numbers-with-AVX-and-AVX
2. Agner Fog (Agner's website): https://www.agner.org

   * Instruction table (latency information): https://www.agner.org/optimize/instruction_tables.pdf
3. x86 and amd64 instruction reference (unofficial) by Félix Cloutier: https://www.felixcloutier.com/x86/
4. Intel Intrinsics Guide: https://software.intel.com/sites/landingpage/IntrinsicsGuide/
5. Computer Organization and Assembly Languages by Yung-Yu Chuang, NTU: https://www.csie.ntu.edu.tw/~cyy/courses/assembly/12fall/news/